In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import sympy as sym

import pycollocation

import inputs
import model
import problem

In [3]:
# define some workers skill
x, loc1, mu1, sigma1 = sym.var('x, loc1, mu1, sigma1')
skill_cdf = 0.5 + 0.5 * sym.erf((sym.log(x - loc1) - mu1) / sym.sqrt(2 * sigma1**2))
skill_params = {'loc1': -1.0, 'mu1': 0.0, 'sigma1': 1.0}
skill_bounds = [-skill_params['loc1'], 1e2]

workers = inputs.Input(var=x,
                       cdf=skill_cdf,
                       params=skill_params,
                       bounds=skill_bounds,
                       )

# define some firms
y, loc2, mu2, sigma2 = sym.var('y, loc2, mu2, sigma2')
productivity_cdf = 0.5 + 0.5 * sym.erf((sym.log(y - loc2) - mu2) / sym.sqrt(2 * sigma2**2))
productivity_params = {'loc2': -1.0, 'mu2': 0.0, 'sigma2': 1.0}
productivity_bounds = [-productivity_params['loc2'], 1e2]

firms = inputs.Input(var=y,
                     cdf=productivity_cdf,
                     params=productivity_params,
                     bounds=productivity_bounds,
                     )

In [4]:
# define symbolic expression for CES between x and y
omega_A, sigma_A = sym.var('omega_A, sigma_A')
A = ((omega_A * x**((sigma_A - 1) / sigma_A) + 
     (1 - omega_A) * y**((sigma_A - 1) / sigma_A))**(sigma_A / (sigma_A - 1))) 

# define symbolic expression for CES between x and y
r, l, omega_B, sigma_B = sym.var('r, l, omega_B, sigma_B')
B = ((omega_B * r**((sigma_B - 1) / sigma_B) + 
     (1 - omega_B) * l**((sigma_B - 1) / sigma_B))**(sigma_B / (sigma_B - 1))) 

F = A * B

In [5]:
# positive assortativity requires that sigma_A * sigma_B < 1
F_params = {'omega_A':0.25, 'omega_B':0.5, 'sigma_A':0.5, 'sigma_B':1.0}

In [6]:
problem = problem.AssortativeMatchingProblem(assortativity='positive',
                                             input1=workers,
                                             input2=firms,
                                             F=sym.limit(F, sigma_B, 1),
                                             F_params=F_params)

In [7]:
solver = pycollocation.OrthogonalPolynomialSolver(problem)

In [10]:
class InitialGuess(object):
    """Class for generating initial coefs for basis functions."""
    
    def _initial_mus(self, assortativity, input1, input2, xs, exp):    
        """
        Guess that mu(x) is a linear transform of some power function. Where
        the intercept and slope of the linear transform are chosen so that
        initial guess satisfies the boundary conditions.

        """
        slope = ((input2.upper - input2.lower) / (input1.upper**exp - input1.lower**exp))
        if assortativity == "positive":
            intercept = input2.lower - slope * input1.lower**exp
        elif assortativity == "negative":
            slope = -slope
            intercept = input2.upper - slope * input1.lower**exp
        else:
            raise ValueError

        return intercept + slope * xs**exp
    
    def _initial_guess_mu(self, xs, mus, deg, domain):
        """Should call out to basis functions!"""
        return np.polynomial.Chebyshev.fit(xs, mus, deg, domain)
    
    def _initial_guess_theta(self, xs, thetas, deg, domain):
        """Should call out to basis functions!"""
        return np.polynomial.Chebyshev.fit(xs, thetas, deg, domain)
    
    def _initial_thetas(self, assortativity, input1, input2, initial_guess_mu):
        """Initial guess for theta(x) should be consistent with mu(x) guess."""        
        H = input1.evaluate_pdf(xs) / input2.evaluate_pdf(initial_guess_mu(xs))
        if assortativity == "positive":
            thetas = (H / initial_guess_mu.deriv()(xs))
        elif assortativity == "negative":
            thetas = -(H / initial_guess_mu.deriv()(xs))
        else:
            raise ValueError
        return thetas

    def compute_initial_coefs(self, assortativity, input1, input2, deg, N=1000, exp=1.0):
        # get domain values
        domain = [input1.lower, input1.upper]
        xs = np.linspace(domain[0], domain[1], N)
        
        # get range values
        mus = self._initial_mus(assortativity, input1, input2, xs, exp)
        thetas = self._initial_thetas(assortativity, input1, input2, xs, exp)
        
        # interpolate using basis functions 
        initial_guess_mu = self._initial_guess_mu(xs, mus, deg, domain)
        initial_guess_theta = self._initial_guess_theta(xs, thetas, deg, domain)
        
        return {'mu': initial_guess_mu.coef, 'theta': initial_guess_theta.coef}

In [11]:
initial_guess = InitialGuess()
initial_coefs = initial_guess.compute_initial_coefs("positive", workers, firms, deg=50, exp=0.5)

AttributeError: 'function' object has no attribute 'deriv'

In [ ]:
# quickly plot the initial conditions
plt.plot(xs, initial_coefs['mu'](xs))
plt.plot(xs, initial_coefs['theta'](xs))
plt.grid('on')

In [ ]:
solver.solve(kind="Chebyshev",
             coefs_dict=initial_coefs,
             domain=domain,
             method='hybr')

In [ ]:
solver.result.success

In [ ]:
viz = pycollocation.Visualizer(solver)

In [ ]:
viz.interpolation_knots = np.linspace(domain[0], domain[1], 1000)
viz.residuals.plot()
plt.show()

In [ ]:
viz.normalized_residuals.plot(logy=True)
plt.show()

In [ ]:
viz.solution.plot(subplots=True)
plt.show()

This is supposed to be the density for firms, but obviously not!

In [ ]:
(viz.solution.theta / workers.evaluate_pdf(viz.solution.index.values)).plot(logy=True)
plt.show()

In [ ]:
solver._coefs_array_to_dict(solver.result.x, solver._degrees)